# 코인 가격 전문가 에이전트

사용자 입력 일자를 기준으로 코인 가격 CSV 데이터(시가/고가/저가/종가, 등락률)를 참고해, 해당 기간의 매수/매도 판단에 대한 구조화된 의견을 제시합니다.

- **입력**: 평가하고 싶은 일자 (예: 2018-03-15)
- **출력**: Pydantic 스키마 (긍정적/보통/부정적, 시장 흐름 분석, 단기·장기 관점, 과거 주목 기간)

## 1. 환경 설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10
python-dotenv could not parse statement starting at line 11
python-dotenv could not parse statement starting at line 12
python-dotenv could not parse statement starting at line 13
python-dotenv could not parse statement starting at line 14
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19


True

In [2]:
# LangSmith 추적 (선택) https://smith.langchain.com
from langchain_teddynote import logging
logging.langsmith("coin-price-expert-agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
coin-price-expert-agent


In [3]:
import pandas as pd
from pathlib import Path
from typing import Literal, List, Dict

from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

## 2. 데이터 로드

CSV 경로는 상수로 두어 추후 API로 교체 시 `load_coin_price_data()`만 수정하면 됩니다.

In [4]:
def get_csv_path() -> Path:
    """노트북과 같은 디렉터리의 CSV 경로 (추후 API로 교체 가능)."""
    return Path("coin_price_index.csv")

def load_coin_price_data(csv_path: Path | None = None) -> pd.DataFrame:
    csv_path = csv_path or get_csv_path()
    df = pd.read_csv(csv_path)
    df["open_time"] = pd.to_datetime(df["open_time"])
    df["date"] = df["open_time"].dt.normalize()
    use_cols = ["date", "open", "high", "low", "close", "price_change_pct"]
    df = df[use_cols].copy()
    # 일자별 중복이 있으면 해당 일자의 마지막 행 사용
    df = df.groupby("date").last().reset_index()
    return df.sort_values("date").reset_index(drop=True)

df = load_coin_price_data()
df.head(10)

,date,open,high,low,close,price_change_pct
0,2018-02-19,11966000.0,12663000.0,11862000.0,12649000.0,0.057078
1,2018-02-20,12649000.0,14149000.0,12645000.0,13548000.0,0.071073
2,2018-02-21,13542000.0,13650000.0,12411000.0,12815000.0,-0.054104
3,2018-02-22,12823000.0,13031000.0,11660000.0,11763000.0,-0.082091
4,2018-02-23,11761000.0,12170000.0,11271000.0,12065000.0,0.025674
5,2018-02-24,12065000.0,12359000.0,11281000.0,11553000.0,-0.042437
6,2018-02-25,11553000.0,11613000.0,10805000.0,11230000.0,-0.027958
7,2018-02-26,11230000.0,11884000.0,10828000.0,11773000.0,0.048353
8,2018-02-27,11778000.0,12165000.0,11463000.0,11951000.0,0.015119
9,2018-02-28,11957000.0,12405000.0,11777000.0,11860000.0,-0.007697


In [5]:
# 기간 추출: 선택된 날짜 이전 6개월 ~ 선택된 날짜 (추후 이 기간 데이터는 API로 받을 예정)
def get_period_data(df: pd.DataFrame, target_date: str, months_before: int = 6) -> str:
    target = pd.to_datetime(target_date)
    start = target - pd.DateOffset(months=months_before)
    end = target
    mask = (df["date"] >= start) & (df["date"] <= end)
    period = df.loc[mask, ["date", "open", "high", "low", "close", "price_change_pct"]].copy()
    def fmt(r):
        d = r["date"].strftime("%Y-%m-%d")
        pct = r["price_change_pct"]
        pct_str = f"{pct:+.2%}" if pd.notna(pct) else "N/A"
        return f"{d}: {r['open']:.0f} | {r['high']:.0f} | {r['low']:.0f} | {r['close']:.0f}, {pct_str}"
    lines = period.apply(fmt, axis=1)
    return "\n".join(lines.tolist())

# 예시 (선택일 이전 6개월 ~ 선택일)
get_period_data(df, "2018-03-15")

'2018-02-19: 11966000 | 12663000 | 11862000 | 12649000, +5.71%\n2018-02-20: 12649000 | 14149000 | 12645000 | 13548000, +7.11%\n2018-02-21: 13542000 | 13650000 | 12411000 | 12815000, -5.41%\n2018-02-22: 12823000 | 13031000 | 11660000 | 11763000, -8.21%\n2018-02-23: 11761000 | 12170000 | 11271000 | 12065000, +2.57%\n2018-02-24: 12065000 | 12359000 | 11281000 | 11553000, -4.24%\n2018-02-25: 11553000 | 11613000 | 10805000 | 11230000, -2.80%\n2018-02-26: 11230000 | 11884000 | 10828000 | 11773000, +4.84%\n2018-02-27: 11778000 | 12165000 | 11463000 | 11951000, +1.51%\n2018-02-28: 11957000 | 12405000 | 11777000 | 11860000, -0.77%\n2018-03-01: 11865000 | 12250000 | 11603000 | 12214000, +2.98%\n2018-03-02: 12214000 | 12581000 | 12123000 | 12465000, +2.06%\n2018-03-03: 12465000 | 12831000 | 12347000 | 12831000, +2.94%\n2018-03-04: 12831000 | 12840000 | 12255000 | 12592000, -1.86%\n2018-03-05: 12592000 | 12899000 | 12451000 | 12746000, +1.22%\n2018-03-06: 12745000 | 12746000 | 11900000 | 12023000,

## 3. Pydantic 응답 스키마 및 Parser

In [6]:
class NotablePeriod(BaseModel):
    title: str = Field(description="기간의 제목 (예: 급락 후 반등 구간)")
    period_text: str = Field(
        description="기간 문자열. 반드시 최대 7일 이내 (예: 2019-02-17 ~ 2019-02-19)"
    )
    period_data: List[Dict[str, str]] = Field(
        description='각 날짜별 가격·등락률. 최대 7개 항목. 각 항목은 날짜(키)와 "시가|고가|저가|종가, 등락률%" 문자열(값). 예: [{"2019-02-17": "11966000|12663000|11862000|12649000, +5.71%"}]',
        max_length=7,
    )

class CoinPriceExpertResponse(BaseModel):
    verdict: Literal["긍정적", "보통", "부정적"] = Field(
        description="해당 일자 매수/매도 판단에 대한 종합 의견 (반드시 세 값 중 하나)"
    )
    market_flow_analysis: str = Field(
        description="해당 기간 시장 흐름 분석: 가격·등락률 추이와 국면 설명"
    )
    short_long_term_perspective: str = Field(
        description="단기적 관점과 장기적 관점을 각각 서술 (예: 단기: 해당 일자 전후 가격·추이, 장기: 6개월 구간에서의 위치·추세)"
    )
    notable_periods: List[NotablePeriod] = Field(
        description="과거 데이터에서 주목할 만한 기간과 그 가격·등락률 (항목 최대 5개, 각 기간은 최대 7일)"
    )

parser = PydanticOutputParser(pydantic_object=CoinPriceExpertResponse)

## 4. 프롬프트, LLM, 체인

In [7]:
prompt = load_prompt("prompts/coin_price_expert.yaml", encoding="utf-8")
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [8]:
MODEL_NAME = "gpt-4.1"
llm = ChatOpenAI(temperature=0, model=MODEL_NAME)
chain = prompt | llm | parser

## 5. 사용자 일자 입력 및 실행

In [9]:
target_date = "2019-02-19"
period_data = get_period_data(df, target_date, months_before=6)

In [10]:
response = chain.invoke({
    "target_date": target_date,
    "period_data": period_data,
})
response

CoinPriceExpertResponse(verdict='보통', market_flow_analysis='2019-02-19는 전일(2/18) 4.85%의 강한 상승 이후, 추가 상승(고가 4338000) 시도가 있었으나 종가는 4259000으로 소폭 상승(+0.54%)에 그쳤습니다. 2/08 이후 단기 급등(5.97%)과 2/18의 강한 상승이 연속적으로 나타났으나, 2/19에는 변동성이 다소 줄고 윗고리가 긴 형태로 마감되어 단기 과열 및 매도세 유입이 감지됩니다. 전반적으로 1월 말~2월 초까지는 횡보 및 약세였으나, 2월 중순 이후 단기 반등세가 뚜렷하게 나타난 구간입니다.', short_long_term_perspective='단기: 2/08 이후 강한 반등세가 이어졌고, 2/18에 대량 상승이 발생한 직후라 단기적으로는 과열 신호와 차익실현 매물 출회 가능성이 높아 보입니다. 2/19 당일은 추가 상승 시도 후 윗고리가 길게 마감되어 단기 매수 진입은 신중해야 할 시점입니다. 장기: 2018년 하반기부터 이어진 하락 추세에서 12월~1월 저점(360만~380만 원대) 이후 바닥권에서 반등이 시도되고 있으나, 전체적으로는 아직 명확한 추세 전환 신호가 부족합니다. 장기적으론 저점 대비 반등 구간이나, 본격적인 상승 전환으로 보기엔 이른 시점입니다.', notable_periods=[NotablePeriod(title='2월 중순 단기 급등 구간', period_text='2019-02-08 ~ 2019-02-19', period_data=[{'2019-02-08': '3783000|4075000|3768000|4009000, +5.97%'}, {'2019-02-09': '4007000|4038000|3968000|3998000, -0.27%'}, {'2019-02-10': '3997000|4030000|3937000|4023000, +0.63%'}, {'2019-02-17': '4002000|4094000|3986000|4040000, +0.95%'

In [11]:
print("=== 종합 의견 (verdict) ===")
print(response.verdict)
print("\n=== 시장 흐름 분석 ===")
print(response.market_flow_analysis)
print("\n=== 단기·장기 관점 ===")
print(response.short_long_term_perspective)
print("\n=== 과거 주목 기간 ===")
for p in response.notable_periods:
    print(f"  - title: {p.title}")
    print(f"    period_text: {p.period_text}")
    print(f"    period_data: {p.period_data}")

=== 종합 의견 (verdict) ===
보통

=== 시장 흐름 분석 ===
2019-02-19는 전일(2/18) 4.85%의 강한 상승 이후, 추가 상승(고가 4338000) 시도가 있었으나 종가는 4259000으로 소폭 상승(+0.54%)에 그쳤습니다. 2/08 이후 단기 급등(5.97%)과 2/18의 강한 상승이 연속적으로 나타났으나, 2/19에는 변동성이 다소 줄고 윗고리가 긴 형태로 마감되어 단기 과열 및 매도세 유입이 감지됩니다. 전반적으로 1월 말~2월 초까지는 횡보 및 약세였으나, 2월 중순 이후 단기 반등세가 뚜렷하게 나타난 구간입니다.

=== 단기·장기 관점 ===
단기: 2/08 이후 강한 반등세가 이어졌고, 2/18에 대량 상승이 발생한 직후라 단기적으로는 과열 신호와 차익실현 매물 출회 가능성이 높아 보입니다. 2/19 당일은 추가 상승 시도 후 윗고리가 길게 마감되어 단기 매수 진입은 신중해야 할 시점입니다. 장기: 2018년 하반기부터 이어진 하락 추세에서 12월~1월 저점(360만~380만 원대) 이후 바닥권에서 반등이 시도되고 있으나, 전체적으로는 아직 명확한 추세 전환 신호가 부족합니다. 장기적으론 저점 대비 반등 구간이나, 본격적인 상승 전환으로 보기엔 이른 시점입니다.

=== 과거 주목 기간 ===
  - title: 2월 중순 단기 급등 구간
    period_text: 2019-02-08 ~ 2019-02-19
    period_data: [{'2019-02-08': '3783000|4075000|3768000|4009000, +5.97%'}, {'2019-02-09': '4007000|4038000|3968000|3998000, -0.27%'}, {'2019-02-10': '3997000|4030000|3937000|4023000, +0.63%'}, {'2019-02-17': '4002000|4094000|3986000|4040000, +0.95%'}, {'2019-02-18': '4040000|4260000